# Lab11-2

## Source1 - CNN_MNIST2(More Deep, Dropout)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32, shape=[None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1]) # img 28*28*1(BW Image)
Y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# Conv -> (?, 28, 28, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
# Pool -> (?, 14, 14, 32)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
# Conv -> (?, 14, 14, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
# Pool -> (?, 7, 7, 64)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# L3 ImgIn shape=(?, 7, 7, 128)
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
# Conv -> (?, 7, 7, 128)
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
# Pool -> (?, 4, 4, 128)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

# reshape = (?, 3136)
L3 = tf.reshape(L3, [-1, 4*4*128])

# L4 fully Connected Layer
W4 = tf.get_variable("W4", shape=[4*4*128, 625], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1, 625]))
L4 = tf.nn.relu(tf.matmul(L3, W4)+b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

# L5 Final FC 625 input -> 10 output
W5 = tf.get_variable("W5", shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1, 10]))

# define Hypothesis
logits = tf.matmul(L4, W5)+b5
hypothesis = tf.nn.softmax(logits)

# define cost
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
cost = tf.reduce_mean(cost_i)

# define gradient
gradient = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# calc accuracy
prediction = tf.argmax(hypothesis, 1)
answer = tf.argmax(Y, 1)
accuracy = tf.reduce_mean( tf.cast( tf.equal(prediction, answer), dtype=tf.float32)) 

# init network
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# go implement
traing_epoch = 15
batch_size = 100

total_recur = 0

start_time = time.time()
for epoch in range(traing_epoch):
    avg_cost = 0.0
    total_batch = int(mnist.train.num_examples/batch_size)
    
    for batch in range(total_batch):
        batch_X, batch_Y = mnist.train.next_batch(batch_size)
        cost_, _ = sess.run([cost, gradient], feed_dict={X: batch_X, Y:batch_Y, keep_prob:0.7})
        avg_cost += cost_/total_batch
        total_recur = total_recur + 1
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

end_time = time.time()
print('Learning Time = ', end_time-start_time)
# implement accuracy
print("Accuracy: ", sess.run(accuracy, feed_dict={X: mnist.test.images, Y:mnist.test.labels, keep_prob:1.0} ) )

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch: 0001 cost = 0.416374410
Epoch: 0002 cost = 0.099702654
Epoch: 0003 cost = 0.076326296
Epoch: 0004 cost = 0.061566270
Epoch: 0005 cost = 0.055902315
Epoch: 0006 cost = 0.048094412
Epoch: 0007 cost = 0.044990774
Epoch: 0008 cost = 0.040231046
Epoch: 0009 cost = 0.038482562
Epoch: 0010 cost = 0.034775018
Epoch: 0011 cost = 0.033566303
Epoch: 0012 cost = 0.031466931
Epoch: 0013 cost = 0.029750194
Epoch: 0014 cost = 0.029086588
Epoch: 0015 cost = 0.025498839
Learning Time =  98.31931710243225
Accuracy:  0.993


[Colaboratory](https://jovianlin.io/pytorch-with-gpu-in-google-colab/)